## Data Indexing and Selection

In our previous module, we looked in detail at methods and tools to access, set, and modify values in NumPy arrays. These included indexing (e.g., `arr[2, 1]`) and slicing (e.g., `arr[:, 1:5]`). Here, we'll look at similar means of accessing and modifying values in Pandas `Series` and `DataFrame` objects, and discuss some other tools like masking and fancy indexing. If you have used the NumPy patterns, the corresponding patterns in Pandas will feel very familiar, though there are a few quirks to be aware of.

We'll start with the simple case of the 1-dimensional `Series` object, and then move onto the more complicated 2-dimensional `DataFrame` object.

### Data Selection in `Series`

As we saw in the previous section, a `Series` object acts in many ways like a 1-dimensional NumPy array, and in many ways like a standard Python dictionary. If we keep these 2 overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### `Series` as a Dictionary

Like a dictionary, the `Series` object provides a mapping from a collection of keys to a collection of values:

In [32]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [4]:
data['b']

0.5

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [5]:
'a' in data

True

In [6]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [7]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

`Series` objects can even be modified with a dictionary-like syntax. Just as you can extend a dictionary by assigning a value to a new key, you can extend a `Series` by assigning a value to a new index value:

In [33]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

### `Series` as a 1-Dimensional Array

A `Series` builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays - that is, *indexing*, *slicing*, and some tools we have not looked at yet, *masking* and *fancy indexing*. Examples of these are as follows:

In [34]:
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

In [9]:
data['c'] # indexing by explicit index

0.75

In [13]:
data[3] # indexing by implicit integer index

1.0

In [11]:
data['a':'e'] # slicing by explicit index

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

In [12]:
data[0:3] # slicing by implicit integer index

a    0.25
b    0.50
c    0.75
dtype: float64

In [14]:
data[(data > 0.3) & (data < 0.8)] # masking

b    0.50
c    0.75
dtype: float64

In [15]:
data[['a', 'e']] # fancy indexing

a    0.25
e    1.25
dtype: float64

Among these, slicing may be the source of the most confusion. Notice that when slicing with an explicit index (i.e., `data['a':'c']`), the final index is *included* in the slide, while when slicing with an implicit index (i.e., `data[0:2]`), the final index is *excluded* from the slice.

### Indexers: `loc` and `iloc`

These slicing and indexing conventions can be a source of confusion. For example, if your `Series` has an explicit integer index, an indexing operation such as `data[1]` will use the explicit indices, while a slicing operation like `data[1:3]` will use the implicit Python-style index:

In [35]:
data = pd.Series(['a', 'b', 'c'],
                index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [17]:
data[1] # explicit index while indexing

'a'

In [18]:
data[1:3] # implicit index while slicing

3    b
5    c
dtype: object

Because of this potential confusion in the case of integer indices, Pandas provides some special *indexer* attributes that explicitly expose certain indexing schemes. These are not functional methods, but attributes that expose a particular slicing interface to the data in the `Series`.

First, the `loc` attribute allows indexing and slicing that always references the explicit index:

In [36]:
data

1    a
3    b
5    c
dtype: object

In [5]:
data.loc[1]

'a'

In [6]:
data.loc[1:3]

1    a
3    b
dtype: object

The `iloc` attribute allows indexing and slicing that always references the implicit Python-style index:

In [7]:
data.iloc[1]

'b'

In [8]:
data.iloc[1:3]

3    b
5    c
dtype: object

One guiding principle of Python code is that "explicit is better than implicit". The explicit nature of `loc` and `iloc` make them very useful in maintaining clean and readable code.

### Data Selection in `DataFrame`

Recall that a `DataFrame` acts in many ways like a 2-dimensional or structured array, and in other ways like a dictionary of `Series` structures sharing the same index. These analogies can be helpful to keep in mind as we explore data selection within this structure.

#### `DataFrame` as a Dictionary

The first analogy we will consider is the `DataFrame` as a dictionary of related `Series` objects. Let's return to our example of areas and populations of states:

In [37]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data2 = pd.DataFrame({'area':area, 'pop':population})
data2

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


The individual `Series` that make up the columns of the `DataFrame` can be accessed via dictionary-style indexing of the column name:

In [10]:
data2['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [38]:
type(data2['area'])

pandas.core.series.Series

Equivalently, we can use attribute-style access with column names that are strings:

In [11]:
data2.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

This attribute-style column access actually accesses the exact same object as the dictionary-style access:

In [12]:
data2.area is data2['area']

True

Though this is a useful shorthand, keep in mind that it does not work for all cases. For example, if the column names are not strings, or if the column names conflict with methods of the `DataFrame`, this attribute-style access is not possible. For example, the `DataFrame` has a `pop()` method, so `data.pop` will point to this rather than the `"pop"` column:

In [ ]:
data2.pop is data['pop'] # this gives an error

In particular, you should avoid the temptation to try column assignment via attribute (i.e., use `data['pop'] = z` rather than `data.pop = z`). 

Like with the `Series` objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case, adding a new column:

In [41]:
data2['density'] = data2['pop'] / data2['area']
data2

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


This shows a preview of the straightforward syntax of element-by-element arithmetic between `Series` objects.

#### `DataFrame` as a 2-Dimensional Array

As mentioned previously, we can also view the `DataFrame` as an enhanced 2-dimensional array. We can examine the raw underlying data array using the values attribute:

In [42]:
data2

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [43]:
data2.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

With this picture in mind, many familiar array-like observations can be done on the `DataFrame` itself. For example, we can transpose the full `DataFrame` to swap rows and columns:

In [17]:
data2.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


When it comes to indexing of `DataFrame` objects, however, it is clear that the dictionary-style indexing of columns precludes the ability to simply treat it as a NumPy array. In particular, passing a single index to an array accesses a row:

In [18]:
data2.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

Passing a single "index" to a `DataFrame` accesses a column:

In [19]:
data2['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Thus, for array-style indexing, we need another convention. Here, Pandas again uses the `loc` and `iloc` indexers mentioned earlier. Using the `iloc` indexer, we can index the underlying array as if it is a simply NumPy array (using the implicit Python-style index), but the `DataFrame` index and column labels are maintained in the result:

In [44]:
data2

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [20]:
data2.iloc[:3, :2]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


Similarly, using the `loc` indexer we can index the underlying data in an array-like style, but using the explicit index and column names:

In [21]:
data2.loc[:'Illinois', :'density']

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


What is the difference between `iloc` and `loc`?

In [23]:
data2.iloc[:5]

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [ ]:
data2.loc[:5] # this gives an error

Any of the familiar NumPy-style data access patterns can be used within these indexers. For example, in the `loc` indexer, we can combine masking and fancy indexing as in the following:

In [25]:
data2.loc[data2.density > 100, ['pop', 'density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


Any of these indexing conventions may also be used to set or modify values; this is done in the standard way that you might be accustomed to from working with NumPy:

In [26]:
data2.iloc[0, 2] = 90
data2

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


### Additional Indexing Conventions

There are a couple extra indexing conventions that might seem at odds with the preceding discussion, but nevertheless can be very useful in practice. First, while *indexing* refers to columns, *slicing* refers to rows:

In [28]:
data2['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Such slices can also refer to rows by number rather than by index:

In [29]:
data2[1:3]

,area,pop,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


Similarly, direct masking operations are also interpreted row-wise rather than column-wise:

In [30]:
data2[data2.area > 170000]

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
Florida,170312,19552860,114.806121


These two conventions are syntactically similar to those of a NumPy array, and while these may not precisely fit the mold of the Pandas conventions, they are nevertheless quite useful in practice.